In [233]:
from importlib import reload
from semantic_search import store
reload(store)
from semantic_search.store import LocalEmbeddingModel

model = LocalEmbeddingModel(
    model_name='allenai/specter2_base',  # 'sentence-transformers/all-MiniLM-L6-v2',
    chunk_size=512,
    chunk_overlap=64,
    batch_size=8
)

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/717k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Using device: cpu


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [234]:
import torch

tmp_input = "Using pooling, it generates from a variable sized sentence a fixed sized sentence embedding. This layer also allows to use the CLS token if it is returned by the underlying word embedding model. You can concatenate multiple poolings together."

inputs = [tmp_input*10, tmp_input*3, tmp_input, tmp_input*30]

_, all_chunks_encoded = model.chunk_and_encode(inputs, progress_bar=True)

# Flatten encoded
tmp = {}
for single_text_encoded in all_chunks_encoded:
    for k, v in single_text_encoded.items():
        if k not in tmp:
            tmp[k] = []
        tmp[k].append(v)
encoded_flattened = {k: torch.cat(v) for k, v in tmp.items()}

# Get embeddings for all chunks
print(f"Generating embeddings for {len(list(encoded_flattened.values())[0])} chunks...")
embeddings = model.get_embeddings(encoded_flattened, progress_bar=True)

Chunking and encoding: 100%|██████████| 4/4 [00:00<00:00, 585.61it/s]


Generating embeddings for 8 chunks...


Generating embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.76s/it]


In [231]:
from torch.nn import functional as F

_, encoded = model.chunk_and_encode(tmp_input*100)
emb = model.get_embeddings(encoded)


encoded_input = model.tokenizer(tmp_input*100, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model.model(**encoded_input)

# Perform pooling
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)



In [232]:
emb[0] == sentence_embeddings[0]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

## Dev for new cleaner store setup

In [10]:
from importlib import reload
from semantic_search import store
reload(store)
from semantic_search.store import LocalEmbeddingModel, FAISSDocumentStore
from tqdm import tqdm

In [11]:
ds = FAISSDocumentStore(db_dir='/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/allenai_specter2_base_test1')
assert ds.load_store()

Using device: cpu
Loaded index with 1016 vectors


In [12]:
ds.document_store.head(5)

,id,doi,ref_via,title,text,type,topic,domain,field,subfield
0,W2194775991,10.1109/cvpr.2016.90,openalex_id,Deep Residual Learning for Image Recognition,Deeper neural networks are more difficult to t...,article,Advanced Neural Network Applications,Physical Sciences,Computer Science,Computer Vision and Pattern Recognition
1,W1686810756,None,openalex_id,Very Deep Convolutional Networks for Large-Sca...,In this work we investigate the effect of the ...,article,Advanced Vision and Imaging,Physical Sciences,Computer Science,Computer Vision and Pattern Recognition
2,W4385245566,10.48550/arxiv.1706.03762,openalex_id,Attention Is All You Need,The dominant sequence transduction models are ...,preprint,Natural Language Processing Techniques,Physical Sciences,Computer Science,Artificial Intelligence
3,W2108598243,10.1109/cvpr.2009.5206848,openalex_id,ImageNet: A large-scale hierarchical image dat...,The explosion of image data on the Internet ha...,article,Advanced Image and Video Retrieval Techniques,Physical Sciences,Computer Science,Computer Vision and Pattern Recognition
4,W2133665775,10.1109/tip.2003.819861,openalex_id,Image Quality Assessment: From Error Visibilit...,Objective methods for assessing perceptual ima...,article,Image and Video Quality Assessment,Physical Sciences,Computer Science,Computer Vision and Pattern Recognition


In [6]:
ds.chunk_store.head(2)

,chunk_id,doc_id,text
0,0,W2194775991,deeper neural networks are more difficult to t...
1,1,W1686810756,in this work we investigate the effect of the ...
